In [1]:
import _init_paths

import cv2
import torch
import numpy as np
import os

from detectors.detector_factory import detector_factory
from datasets.dataset_factory import get_dataset
from models.decode import ddd_decode
from utils.debugger import Debugger
from opts import opts

from matplotlib import pyplot as plt

In [2]:
from utils.image import transform_preds
from trains.train_factory import train_factory
from models.model import create_model, load_model, save_model

In [4]:
from models.utils import _transpose_and_gather_feat
import torch.nn.functional as F

In [5]:
MODEL_PATH = '../models/BIG_500K_full_3d.pth'
MODEL_PATH = '../exp/ddd/fish_val/model_last.pth'
TASK = 'fish3d'
DATASET = 'kitti'
IMG_PATH = '../data/kitti/images/trainval/000079.jpg'

# Dataset Loader

In [6]:
Dataset = get_dataset('fish_sim', 'fish')

opt = opts().init(['fish','--exp_id','3dop','--dataset','fish_sim','--kitti_split','3dop','--batch_size','8','--num_epochs','70','--lr_step','45,60','--gpus','0'])
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)

Fix size testing.
training chunk_sizes: [8]
The output will be saved to  /home/alfin/Documents/deep_learning/production/centernet_3d_fish/src/lib/../../exp/fish/3dop
heads {'hm': 1, 'reg': 2, 'dep': 1, 'dim': 3, 'rot': 4, 'wh': 2, 'reid': 128}
heads {'hm': 1, 'reg': 2, 'dep': 1, 'dim': 3, 'rot': 4, 'wh': 2, 'reid': 128}


In [7]:
ds = Dataset(opt, 'val')

train_loader = torch.utils.data.DataLoader(
    ds, 
    batch_size=1, 
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

==> initializing fish 3dop, val data.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loaded val 300 samples


# Model 

In [8]:
detector = detector_factory[opt.task](opt)
debugger = Debugger(dataset=detector.opt.dataset, ipynb=(detector.opt.debug==3),
                        theme=detector.opt.debugger_theme)

Creating model...


FileNotFoundError: [Errno 2] No such file or directory: ''

In [8]:
# model = detector.model
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)

Trainer = train_factory[TASK]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

# LOSS ANALYSIS

In [9]:
for key in ds[0].keys():
    if type(ds[0][key]) != dict:
        print(key,ds[0][key].shape,type(ds[0][key])) 

input (3, 512, 512) <class 'numpy.ndarray'>
hm (3, 128, 128) <class 'numpy.ndarray'>
ind (40,) <class 'numpy.ndarray'>
dep (40, 1) <class 'numpy.ndarray'>
dim (40, 3) <class 'numpy.ndarray'>
rotX (40, 1) <class 'numpy.ndarray'>
rotY (40, 1) <class 'numpy.ndarray'>
reg_mask (40,) <class 'numpy.ndarray'>
reg (40, 2) <class 'numpy.ndarray'>


In [10]:
data = ds[0]

In [12]:
img_input, meta = detector.pre_process(np.moveaxis(data['input'], 0, -1), detector.scales)
img_input = img_input.to(detector.opt.device)
output = detector.model(img_input)[-1]

output['hm'] = output['hm'].sigmoid_()
output['dep'] = 1. / (output['dep'].sigmoid() + 1e-6) - 1.
# wh = output['wh'] if detector.opt.reg_bbox else None
reg = output['reg'] if detector.opt.reg_offset else None
torch.cuda.synchronize()

output.keys()

dict_keys(['hm', 'dep', 'rotX', 'rotY', 'dim', 'reg'])

In [12]:
num_heading_bin = 12

# converting dataset
mask = torch.Tensor(np.expand_dims(np.array(data['rot_mask']), axis = 1))
target_bin = torch.Tensor(data['heading_binX']).type(torch.long)
target_res = torch.Tensor(data['heading_resX'])

# converting outputs
headingX = output['headingX'].detach().cpu()
ind = torch.Tensor(np.expand_dims(np.array(data['ind']), axis = 0)).type(torch.int64)
headingX = _transpose_and_gather_feat(headingX, ind)

# reshape
headingX = headingX.view(-1, num_heading_bin*2)
target_bin = target_bin.view(-1, 1)
target_res = target_res.view(-1, 1)

print(headingX.shape,target_bin.shape,target_res.shape,mask.shape)

KeyError: 'headingX'

In [13]:
heading_input_cls = headingX[:, 0:num_heading_bin]
heading_input_cls[0]

NameError: name 'headingX' is not defined

In [137]:
# extract important feature
mask_cls = mask.expand_as(heading_input_cls).float()
heading_input_cls = heading_input_cls*mask_cls

In [156]:
F.cross_entropy(heading_input_cls, target_bin[:,0], reduction='mean')

tensor(2.4855)

In [157]:
target_bin[:,0]

tensor([1, 2, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [159]:
heading_input_cls.shape

torch.Size([100, 12])